In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('/Users/nikolaiwegge-berg/Desktop/1. semester/Social graphs/Exam social graph/representatives.csv')

# Define a function to clean the "Age" column
def clean_age(age_str):
    if isinstance(age_str, str):
        age_str = re.findall(r'\d+', age_str)  # Remove non-numeric characters
        if age_str:
            return int(age_str[0])  # Convert the cleaned string to an integer
    return None

# Apply the function to the "Age" column
df['Age'] = df['Age'].apply(clean_age)

# Save the cleaned DataFrame to a new CSV file
df.to_csv('cleaned_file.csv', index=False)

In [4]:
# Read the CSV file into a DataFrame
df = pd.read_csv('/Users/nikolaiwegge-berg/Desktop/1. semester/Social graphs/02805_SocialGraphs/usa_data/senators.csv')

# Define a function to clean the "Age" column
def clean_age(age_str):
    if isinstance(age_str, str):
        age_str = re.findall(r'\d+', age_str)  # Remove non-numeric characters
        if age_str:
            return int(age_str[0])  # Convert the cleaned string to an integer
    return None

# Apply the function to the "Age" column
df['Age'] = df['Age'].apply(clean_age)

# Save the cleaned DataFrame to a new CSV file
df.to_csv('Senators.csv', index=False)

In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_current_members_of_the_United_States_House_of_Representatives'
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text,"lxml")

house_of_reps_text = soup.find('table',attrs={'class':"wikitable sortable plainrowheaders", 'id': "votingmembers"})

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
representatives = pd.read_html(str(house_of_reps_text))[0]
representatives = representatives.drop(['Party'], axis=1)
representatives = representatives.rename(columns={'Party.1': 'Party', 'Born[4]': 'Age'})
print(representatives.columns)

Index(['District', 'Member', 'Party', 'Prior experience', 'Education',
       'Assumed office', 'Residence', 'Age'],
      dtype='object')


In [ ]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_current_United_States_senators?fbclid=IwAR3fwBqo12plcIm741lp9eP2ALJ4G49Ntch2P9_Dxg8cOmiQKODTrPluMT8'
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text,"lxml")

senators_text = soup.find('table',attrs={'class':"wikitable sortable", 'id': "senators"})
senators = pd.read_html(str(senators_text))[0]
senators = senators.drop(['Party', 'Portrait'], axis=1)
senators = senators.rename(columns={'Party.1': 'Party', 'Residence[5]': 'Residence', 'Born': 'Age'})
# save to csv


In [ ]:
# Save to csv 
representatives.to_csv('representatives.csv', index=False)
senators.to_csv('senators.csv', index=False)

In [ ]:
senators= pd.read_csv('senators.csv')
representatives = pd.read_csv('representatives.csv')
representatives = representatives[representatives["Member"] != "Vacant"]

In [ ]:
def clean_name(name):
    name = name.replace('"', '')
    return unidecode(name)

In [ ]:
house_names = representatives['Member'].tolist()
senators_names = senators['Senator'].tolist()

In [ ]:
import os
def get_page_content(pol_name):
    params = {
        "action": "query",
        "prop": "extracts",
        "exlimit":"1",
        "explaintext": "1",
        "format": "json",
        "titles": pol_name  
    }
    baseurl = "https://en.wikipedia.org/w/api.php?"
    # requests version
    wikitext = requests.get(baseurl, params=params)
    wikijson = wikitext.json()

    # get page id
    pageid = list(wikijson['query']['pages'].keys())[0]
    #wikitext = wikijson['query']['pages'][pageid]['revisions'][0]['*']
    wikitext = wikijson['query']['pages'][pageid]['extract']
    # use regex to count words in wikitext

    return wikitext

if not os.path.isdir('usa_data/senate'):
    os.mkdir('usa_data/senate')

if not os.path.isdir('usa_data/house'):
    os.mkdir('usa_data/house')

In [ ]:
import re
import requests
import string

for name in senators_names:
    wikitext = get_page_content(name)
    with open('usa_data/senate/{}.txt'.format(name), 'w') as f:
        f.write(wikitext)


for name in house_names:
    wikitext = get_page_content(name)
    name = re.sub(r'[".?"]','',name)
    with open('usa_data/house/{}.txt'.format(name), 'w') as f:
        f.write(wikitext)

In [ ]:
def getLinks(rapper_name):
    params = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "format": "json",
    "titles": rapper_name
    
    }
    baseurl = "https://en.wikipedia.org/w/api.php?"
    # requests version
    wikitext = requests.get(baseurl, params=params)
    wikijson = wikitext.json()


    pageid = list(wikijson['query']['pages'].keys())[0]
    wikitext = wikijson['query']['pages'][pageid]['revisions'][0]['*']
    links = re.findall(r'\[\[(.*?)\]\]', wikitext)
    links = [link.split('|')[0] for link in links]
    links = [link for link in links if link in senators_names or link in house_names]
    # keep only unique links
    links = list(set(links))
    return links

In [ ]:
house_links = dict()
senate_links = dict()
counter = 0

def getPairs(dict1,Names):
    for pol in Names:
        links = getLinks(pol)
        links = [clean_name(link) for link in links]
        dict1[clean_name(pol)] = []
        dict1[clean_name(pol)] += links

getPairs(house_links,house_names)
getPairs(senate_links,senators_names)

['Andrew Clyde', 'Kat Cammack']
[]
[]
['Steve Scalise', 'Katie Britt', 'Earl Blumenauer', 'Kevin McCarthy']
['Emilia Sykes', 'Kevin McCarthy']
[]
['Kirsten Gillibrand', 'Nancy Pelosi']
['Tom Cole', 'Markwayne Mullin', 'Jim Costa', 'Lisa Murkowski', 'Jared Golden', 'Sharice Davids', 'Marie Gluesenkamp Perez', 'Brad Finstad']
['Martin Heinrich', 'Paul Gosar', 'Juan Ciscomani', 'Andy Biggs']
['Jasmine Crockett', 'Joyce Beatty', 'Kevin McCarthy']
['Tom Emmer', 'Garret Graves', 'Raul Grijalva', 'Kyrsten Sinema', 'Eric Swalwell']
['Pete Stauber', 'Abigail Spanberger', 'Kyrsten Sinema', 'Paul Gosar']
['Ken Buck', 'Matt Gaetz', 'Jack Bergman', 'Lisa Blunt Rochester', 'Jim Jordan', 'Kevin McCarthy', 'Paul Gosar']
['Lori Chavez-DeRemer', 'David Schweikert', 'Kevin McCarthy']
['Barbara Lee', 'Mario Diaz-Balart', 'Bernie Sanders', 'Mark Pocan', 'Ruben Gallego', 'Pramila Jayapal', 'Bruce Westerman']
['Nancy Pelosi', 'Michael Cloud']
['Ted Cruz', 'Mitch McConnell', 'Greg Stanton', 'Chuck Fleischmann

In [ ]:
from unidecode import unidecode

def clean_name(name):
    name = name.replace('"', '')
    return unidecode(name)

house_links_copy = {}
senate_links_copy = {}
#copy congress_dict
# loop through the dictionary and clean the names
for key, value in house_links.items():
    key = clean_name(key)
    house_links_copy[clean_name(key)] = [clean_name(name) for name in value]

for key, value in senate_links_copy.items():
    key = clean_name(key)
    senate_links_copy[clean_name(key)] = [clean_name(name) for name in value]

senate_links = senate_links_copy
house_links = house_links_copy

In [ ]:
# save house names to json
import json

with open('usa_data/house_links.json', 'w') as f:
    json.dump(house_links, f)

with open('usa_data/senate_links.json', 'w') as f:
    json.dump(senate_links, f)

In [ ]:
# create csv with name and house or senate
all_list = []

# delete file if exists
if os.path.exists('usa_data/all_list.csv'):
    os.remove('usa_data/all_list.csv')

for name in house_names:
    with open('usa_data/all_list.csv', 'a') as f:
        f.write(str(name + "," + "house"+ "\n"))
        f.close()


for name in senators_names:
    with open('usa_data/all_list.csv', 'a') as f:
        f.write(str(name + "," + "senate"+ "\n"))
        f.close()
